## R-alpha

In [1]:
!pip install smartapi-python
!pip install websocket-client
!pip install logzero
!pip install pyotp
!pip install requests
!pip install numpy
!pip install pandas
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.6 MB/s eta 0:00:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 20.2 MB/s eta 0:00

In [2]:
# package import statement
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
from connect_api import connect_api, create_api_session
from data import get_data_stream, token_lookup, get_instrument_list, hist_data, hist_data_from_now
from strategy import expected_buy_price, expected_sell_price, trend_detection, exit_long, exit_short
from order import place_limit_order, place_market_order
from momentum import get_top_nse_stocks_by_volume, momentum_strategy
import json
import threading
import time   

In [3]:
with open('credentials.json', 'r') as file:
    credentials = json.load(file)

api_key = credentials['api_key']
token_id = credentials['token_id']
username = credentials['username']
pwd = credentials['pwd']

In [4]:
smartApi = connect_api(api_key)
data = create_api_session(token_id, username, pwd, smartApi)

[I 240529 08:55:24 smartConnect:121] in pool


### Get Ticker Data

In [9]:
instrument_list = get_instrument_list()

ticker_list = ["BANKNIFTY29MAY2450900CE"]
token_list_0 = []

for ticker in ticker_list:
    token = token_lookup(ticker, instrument_list, exchange="NFO")
    token_list_0.append(token)

token_list = [
        {
            "exchangeType": 2,
            "tokens": token_list_0
        }
    ]


print(token_list)

[{'exchangeType': 2, 'tokens': ['56695']}]


## Create Live Data Stream

In [10]:
correlation_id = "stream_2" #any string value which will help identify the specific streaming in case of concurrent streaming
action = 1 #1 subscribe, 0 unsubscribe
mode = 1 #1 for LTP, 2 for Quote and 3 for SnapQuote
    
sws = get_data_stream(token_id, api_key, username, smartApi.getfeedToken(), token_list, correlation_id, action, mode)

## Momentum Strategy

In [11]:
import pandas as pd

ticker_ltp = {}
position = 0
momentum_short_window = 100
momentum_long_window = 200
trades = {}
capital = 1500
symbol = 'ITC-EQ'

ltp_list = []

def momentum_strategy(tick, exchange):

    global position
    global trades
    global capital
    global ticker_ltp
    global symbol
    global ltp_list
    ### add ltp and return to dictionary
    # token = tick['token']
    ltp = tick['last_traded_price']/100
    
    ltp_list.append(ltp)

    print(ltp)

#     momentum_signal = 0

#     if token in ticker_ltp:
#         ticker_ltp[token]['ltp'].append(ltp)
        
#         momentum_short = pd.Series(ticker_ltp[token]['ltp']).rolling(window=momentum_short_window).mean().iloc[-1]
#         ticker_ltp[token]['momentum_short'].append(momentum_short)
        
#         momentum_long = pd.Series(ticker_ltp[token]['ltp']).rolling(window=momentum_long_window).mean().iloc[-1]
#         ticker_ltp[token]['momentum_long'].append(momentum_long)
#     else:
#         ticker_ltp[token] = {
#             'ltp': [ltp],
#             'momentum_short':[None],
#             'momentum_long': [None]
#         }


#     # Check momentum signal
#     if (ticker_ltp[token]['momentum_short'][-1] is not None) and (ticker_ltp[token]['momentum_long'][-1] is not None):
#         if ticker_ltp[token]['momentum_short'][-1] > ticker_ltp[token]['momentum_long'][-1]:
#             momentum_signal = 1
#         elif ticker_ltp[token]['momentum_short'][-1] < ticker_ltp[token]['momentum_long'][-1]:
#             momentum_signal = -1

#     #create trade

#     if momentum_signal == 1 and position == 0:
#         quantity = capital // ltp
#         price = ltp
#         position = 1
#         capital = capital - quantity*price

#         if token in trades:
#             trades[token]['buy_prices'].append(price)
#             trades[token]['quantity'].append(quantity)
#             # trades[token]['sell_price'].append(None)
        
#         else:
#             trades[token] = {
#             'buy_prices': [price],
#             'quantity': [quantity],
#             'sell_price': []
#             }

#         place_limit_order(instrument_list,symbol,token,'BUY',price,quantity,smartApi, exchange=exchange)

#     elif momentum_signal == 1 and position == -1:
#         quantity = trades[token]['quantity'][-1]
#         price = ltp
#         position = 0
#         capital = capital + quantity*price
        
#         if token in trades:
#             trades[token]['buy_prices'].append(price)
        
#         else:
#             trades[token] = {
#             'buy_prices':price
#             }

#         place_limit_order(instrument_list,symbol,token,'BUY',price,quantity,smartApi, exchange=exchange)


#     elif momentum_signal == -1 and position == 0:
#         quantity = -capital // ltp
#         price = ltp
#         position = -1
#         capital = capital + quantity*price
        
#         if token in trades:
#             trades[token]['sell_price'].append(price)
        
#         else:
#             trades[token] = {
#             'sell_price':price
#             }

#         place_limit_order(instrument_list,symbol,token,'SELL',price,-quantity,smartApi, exchange=exchange)

#     elif momentum_signal == -1 and position == 1:
#         quantity = trades[token]['quantity'][-1]
#         price = ltp
#         position = 0

#         capital = capital + quantity*price
#         if token in trades:
#             trades[token]['sell_price'].append(price)
        
#         else:
#             trades[token] = {
#             'sell_price':price
#             }

#         place_limit_order(instrument_list,symbol,token,'SELL',price,quantity,smartApi, exchange=exchange)


def on_data(ws, message):
    momentum_strategy(message,exchange='NFO')

sws.on_data = on_data

In [12]:
sws.connect()

[I 240529 08:58:46 data:99] on open


0.6
0.6
0.6
0.6
0.6
0.6
0.6
0.6
0.65
0.65
0.65
0.65
0.65
0.6
0.6
0.6
0.6
0.6
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.6
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.6
0.55
0.55
0.55
0.6
0.6
0.55
0.55
0.6
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.6
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.6
0.6
0.6
0.6
0.6
0.6
0.6
0.55
0.55
0.55
0.6
0.6
0.55
0.6
0.6
0.6
0.6
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.6
0.6
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.6
0.6
0.6
0.6
0.6
0.6
0.6
0.6
0.6
0.6
0.55
0.55
0.6
0.6
0.6
0.6
0.6
0.6
0.6
0.55
0.55
0.6
0.6
0.6
0.6
0.6
0.6
0.6
0.6
0.55
0.55
0.6
0.6
0.6
0.6
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.6
0.6
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.6
0.6
0.55
0.55
0.55
0.55
0.55
0.6
0.6
0.6
0.6
0.6
0.55
0.55
0.55
0.6
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.55
0.5
0.5
0.55
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.55
0.55
0.55
0.55
0.5
0.5
0.5
0.5
0.5
0.5
0.5

[W 240529 09:46:57 smartWebSocketV2:319] Attempting to resubscribe/reconnect (Attempt 1)...
[W 240529 09:47:02 smartWebSocketV2:319] Attempting to resubscribe/reconnect (Attempt 2)...


KeyboardInterrupt: 

In [14]:
len(ltp_list)

4681

## misc

In [ ]:
correlation_id = "stream_1"
action = 1
mode = 1
token_list = [
        {
            "exchangeType": 1,
            "tokens": ["26009"]
        }
    ]

sws = get_data_stream(token_id, api_key, username, smartApi.getfeedToken(), token_list, correlation_id, action, mode)

In [ ]:
# Create a new thread and pass the function to it
my_thread = threading.Thread(target=sws.connect())

# Start the thread
my_thread.start()
time.sleep(10)

In [ ]:
place_market_order(instrument_list,ticker="ITC",buy_sell="BUY",price=0,quantity=1,obj=smartApi, sl=0,sqof=0,exchange="NSE")

In [ ]:
nifty_50 = hist_data(smartApi, "NIFTY23MAY2422500CE",st_date,end_date,interval,instrument_list,exchange="NSE"

In [ ]:
nifty_50 = hist_data_from_now(smartApi, "NIFTY23MAY2422500CE", 1, "ONE_MINUTE", instrument_list, exchange="NFO")

In [ ]:
nifty_50.to_csv("niftybees_2000day_1day.csv")

In [ ]:
import pandas as pd

params = {
             "exchange": "NFO",
             "symboltoken": "38764",
             "interval": "ONE_MINUTE",
             "fromdate": "2024-04-22 09:15",
             "todate": "2024-05-22 15:30"
             }
hist_data = smartApi.getCandleData(params)
df_data = pd.DataFrame(hist_data["data"],
                       columns = ["date","open","high","low","close","volume"])
df_data.set_index("date",inplace=True)

In [ ]:
df_data.to_csv("nifty_ce_

In [15]:
df_ltp = pd.DataFrame(ltp_list, columns = ['NIFTY_OUT_OF_MONEY_LAST_DAY_LTP_45MIN'])


In [16]:
df_ltp.to_csv('NIFTY_OUT_OF_MONEY_LAST_DAY_LTP_45MIN.csv', index=False)